# 한총 장바구니 분석

In [1]:
#데이터셋 가져오기
#order 컬럼명 바꾸기 - 숫자로 되어 있는 부분에 Order (purchase event) 입력
import pandas as pd
#카테고리 전처리
import re
# 장바구니 분석
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import apriori, association_rules
import datetime as dt
import os

### 0. 경로 설정

In [2]:
input_path = 'C:/GMG_R&D/3. 장바구니 분석/1. 한총'  #input("작업 파일 경로를 입력하세요")
os.chdir(input_path)
path=os.getcwd()
print(path)

#폴더 생성
x = dt.datetime.now()
file_name=f'분석결과_1depth_{x.year}{x.month}{x.day}'

if not os.path.exists(file_name):
    os.mkdir(file_name) 

C:\GMG_R&D\3. 장바구니 분석\1. 한총


### 1. 한총 데이터 로드

In [6]:
df= pd.read_clipboard(sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50001 entries, Model Code (v41) to EF-RS906CNEGKR,EP-TA800XWKGKR
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   2020년            50001 non-null  int64
 1   2021년            50001 non-null  int64
 2   2022.01~2022.11  50001 non-null  int64
dtypes: int64(3)
memory usage: 1.5+ MB


In [7]:
df=df[1:]
df=df.reset_index().rename(columns={"index": "Model Code (v41)"})
df

,Model Code (v41),2020년,2021년,2022.01~2022.11
0,EP-DG930MBKGKR,9746,6236,3665
1,AA-BC7AU5B,9365,0,0
2,SM-R175NZWAKOO,4961,344,0
3,SM-N986NZNEKOO,4950,172,0
4,SM-R175NZBAKOO,4906,127,0
...,...,...,...,...
49995,"EJ-P5450SBEGKR,GP-TTT870KDATK,GP-TTT870KDATK",0,0,1
49996,"1FF10014,1FF10014,1FF10014,SAM20220221001,SAM2...",0,0,1
49997,"V00073,V00225,V00235",0,0,1
49998,"21,405,702,145,135",0,0,1


In [8]:
df['Order (purchase event)']=df.iloc[:,1:4].sum(axis=1)
df

,Model Code (v41),2020년,2021년,2022.01~2022.11,Order (purchase event)
0,EP-DG930MBKGKR,9746,6236,3665,19647
1,AA-BC7AU5B,9365,0,0,9365
2,SM-R175NZWAKOO,4961,344,0,5305
3,SM-N986NZNEKOO,4950,172,0,5122
4,SM-R175NZBAKOO,4906,127,0,5033
...,...,...,...,...,...
49995,"EJ-P5450SBEGKR,GP-TTT870KDATK,GP-TTT870KDATK",0,0,1,1
49996,"1FF10014,1FF10014,1FF10014,SAM20220221001,SAM2...",0,0,1,1
49997,"V00073,V00225,V00235",0,0,1,1
49998,"21,405,702,145,135",0,0,1,1


In [ ]:
for i in df.columns:
    if i=='Model Code (v41) (evar41)': 
        df=df.rename(columns={'Model Code (v41) (evar41)':'Model Code (v41)'})
df.columns

### 2개 이상 구매자 비중

In [9]:
for percent_df in df.columns[1:]:
    print('****************************')
    print(percent_df )
    all=df[percent_df].sum()
    print('전체: ',all)
    print('2개이상: ',df[df[['Model Code (v41)',percent_df]]['Model Code (v41)'].str.contains(',')][percent_df].sum())
    
    print("제품 2개 구매 비중: ",round((df[df[['Model Code (v41)',percent_df]]['Model Code (v41)'].str.contains(',')][percent_df].sum()/all)*100,2))


****************************
2020년
전체:  365380
2개이상:  63825
제품 2개 구매 비중:  17.47
****************************
2021년
전체:  738544
2개이상:  81837
제품 2개 구매 비중:  11.08
****************************
2022.01~2022.11
전체:  842619
2개이상:  58436
제품 2개 구매 비중:  6.94
****************************
Order (purchase event)
전체:  1946543
2개이상:  204098
제품 2개 구매 비중:  10.49


In [10]:
df=df[df['Model Code (v41)'].str.contains(',')]
print('콤마 들어간거 행 수:',df.shape[0])
df=df.reset_index(drop=True)

콤마 들어간거 행 수: 41987


In [11]:
#전체 SKU 하나의 리스트로
a = df['Model Code (v41)'].tolist()
b = ','.join(a) #리스트 문자열로
c = b.split(',') #문자열 개별 문자로 리스트화 
model = list(set(c)) #중복 제거

### 2. 카테고리 파일 로드

In [12]:
#카테고리 가져오기
rex= pd.read_clipboard(sep='\t')
rex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Category1D        7 non-null      object
 1   Category1D_Value  7 non-null      object
dtypes: object(2)
memory usage: 240.0+ bytes


In [13]:
#전체 SKU 담을 딕셔너리
dic_category = {}

for m in model :
    flag = 1
    for i in range(len(rex)):
        r = re.compile(rex.Category1D_Value[i])
        if r.search(m):
            dic_category[m] = rex.Category1D[i]
            flag = 0
            continue
    if flag :
        dic_category[m] = '기타'

In [14]:
dic_category 

{'': '기타',
 'UN43N5020AFXKR': 'TV',
 '184': '기타',
 'SWA-W510/KR': '오디오',
 'EF-DX900UBKGKR': '모바일',
 'MR-100103': '기타',
 'GP-TFN975KDATK': '모바일',
 'D35192': '기타',
 'EF-ZX700PWEGKR': '모바일',
 '107': '기타',
 'RA-F17NUU39': '주방가전',
 'SM-F936NZEFKOO': '모바일',
 'G91110006': '기타',
 'AKGY100BTP': '오디오',
 'P0000CYW': '기타',
 'V00151': '기타',
 'EF-XA336CBEGKR': '모바일',
 'EF-KG980CLEGKR': '모바일',
 'MUVC-1F7110237': '모바일',
 'OTTA0010': '기타',
 '767': '기타',
 'dodram10494': '기타',
 'SF241772': '기타',
 'F0007': '기타',
 'RA-F17DUU06': '주방가전',
 'RA-R23DAAGLLD1': '주방가전',
 'DW60B8155LWS': '주방가전',
 'SKK-AKP': '생활가전',
 'RA-D24DMM32': '주방가전',
 'JBLGO3RED': '오디오',
 'NT950XDC-XD71S': 'PC/주변기기',
 '35548': '기타',
 'SAM20220915031': '기타',
 'WF24T8000KV': '생활가전',
 'SAMEVENT20220105': '기타',
 'ET-SFR86SYEGKR': '모바일',
 'RA-F18DUU05': '주방가전',
 'JBLLIVEFREE2TWSROS': '오디오',
 'ET-SKR82MJEGKR': '모바일',
 'HGFG0028': '기타',
 'EF-EA536PWEGKR': '모바일',
 'DW50R4055BBS': '주방가전',
 'sf244748': '기타',
 'AC100BS4DBC1SY': '생활가전',
 'VR50T95936U': '

In [15]:
#데이터셋 만들기
dataset = []    # 데이터셋 구조 [ [a,b], [a,c], [a,b,c] .... ]

for i in range(len(df)) :
    sku = df['Model Code (v41)'][i]
    revised = sku.split(",")
    code = []
    for j in range(len(revised)) :
        code.append(dic_category[revised[j]])   #각 SKU 리스트에 담기
    
    for k in range(df['Order (purchase event)'][i]) :   #order 수만큼 담기
        dataset.append(code) 
    

### 1) 기타 제거

In [ ]:
 #기타제거
ho = []
rm_set = {'기타'}
for s in dataset:
    arr_new = [i for i in s if i not in rm_set]
    ho.append(arr_new) # [5, 7, 2, 7]
dataset=ho

print('기타 제거 후, 행 수: ',len(dataset))

원래 행 수:  199894
기타 제거 후, 행 수:  199894


In [17]:
#장바구니 분석
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset) #True / False
df2 = pd.DataFrame(te_ary, columns=te.columns_)
itemsets_data = apriori(df2, min_support=0.00001, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
#rules = association_rules(itemsets_data, metric='confidence', min_threshold=1).sort_values('confidence',ascending=False).reset_index(drop=True)
itemsets_data['count']=(itemsets_data['support']	*len(dataset)).astype(int)
itemsets_data.to_csv(f'{path}\\{file_name}\\{x.month}월_한총 분석결과파일 1depth.csv', encoding = 'utf-8-sig')
itemsets_data

,support,itemsets,count
0,0.732761,(모바일),149555
1,0.097801,(생활가전),19961
2,0.066135,(PC/주변기기),13498
3,0.064939,(기타),13254
4,0.033508,(주방가전),6839
5,0.017070,(오디오),3483
6,0.015576,(TV),3179
7,0.014385,"(주방가전, 생활가전)",2936
8,0.008486,(smartthings),1732
9,0.005238,"(모바일, 오디오)",1069


In [75]:
'''   association_rules을 진행한다면 하기 

x = dt.datetime.now()

num_conq=1
N=4

def frozen_convert(sets) : 
    return [list(sets)]

for num_ant in [1,2,3]:
    df = rules[rules.apply(lambda x:True if len(x.antecedents) == int(num_ant) else False, axis=1) & rules.apply(lambda x:True if len(x.consequents) == int(num_conq) else False, axis=1)]
    df = df.sort_values('confidence', ascending=False)
    df = df.round(N)
    df[['antecedents','consequents']] =df[['antecedents','consequents']].applymap(lambda x: frozen_convert(x))
    df['antecedents']=df['antecedents'].astype(str).apply(lambda x: x.replace('[[','['))
    df['antecedents']=df['antecedents'].apply(lambda x: x.replace(']]',']'))
    df['consequents']=df['consequents'].astype(str).apply(lambda x: x.replace('[[','['))
    df['consequents']=df['consequents'].apply(lambda x: x.replace(']]',']'))
    df.to_csv(f'{path}\\{file_name}\\{x.month}월_한총 분석결과파일_{num_ant}.csv', encoding = 'utf-8-sig')

'''